## Save Roadnet Coords

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pickle
import osmnx as ox

<IPython.core.display.Javascript object>

In [3]:
import sys

sys.path.append("/app")
from constants import DATA_PATH

<IPython.core.display.Javascript object>

In [4]:
import os

<IPython.core.display.Javascript object>

In [5]:
from db_utils import get_engine, get_table_creation_query

<IPython.core.display.Javascript object>

In [6]:
import logging

<IPython.core.display.Javascript object>

## Connect to Database

In [7]:
user = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
host = os.getenv("POSTGRES_HOST")

<IPython.core.display.Javascript object>

In [8]:
engine = get_engine(user, password, host)

<IPython.core.display.Javascript object>

In [9]:
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter("%(asctime)s [%(name)s] %(levelname)-8s %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

<IPython.core.display.Javascript object>

In [10]:
logger.info("Attempting to load network...")
try:
    with open(os.path.join(DATA_PATH, "roadnet_greaterlondon_walking.pkl"), "rb") as f:
        walknet = pickle.load(f)
        logger.info("Loaded network")
except EOFError as e:
    logger.warning("unable to load from pickle. Loading from graphml")
    walknet = ox.load_graphml(
        os.path.join(DATA_PATH, "roadnet_greaterlondon_walking.graphml")
    )
    logger.info("saving new network")
    with open(os.path.join(DATA_PATH, "roadnet_greaterlondon_walking.pkl"), "wb") as f:
        walknet = pickle.dump(walknet, f)
    logger.info("saved new network as pickle")

2021-05-27 10:31:45,752 [root] INFO     Attempting to load network...
Exception ignored in: <function BaseGeometry.__del__ at 0x7f1a2bfeb280>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/shapely/geometry/base.py", line 241, in __del__
    def __del__(self):
KeyboardInterrupt: 
Exception ignored in: <function BaseGeometry.__del__ at 0x7f1a2bfeb280>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/shapely/geometry/base.py", line 241, in __del__
    def __del__(self):
KeyboardInterrupt: 
Exception ignored in: <function BaseGeometry.__del__ at 0x7f1a2bfeb280>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/shapely/geometry/base.py", line 242, in __del__
    self.empty(val=None)
  File "/usr/local/lib/python3.8/site-packages/shapely/geometry/base.py", line 227, in empty
    self._lgeos.GEOSGeom_destroy(self.__geom__)
KeyboardInterrupt: 
Exception ignored in: <function BaseGeometry.__del__

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

In [ ]:
%%time
gdf_nodes, _ = ox.graph_to_gdfs(walknet)


In [ ]:
cols = {
    "eastings": "DECIMAL(14,6)",
    "northings": "DECIMAL(14,6)",
    "osmid": "BIGINT",
}

index_cols = ["osmid"]
unique_cols = ["osmid"]

In [ ]:
q_create = get_table_creation_query(
    "node_coords", cols, "public", index_cols, unique_cols
)

In [ ]:
with engine.connect() as conn:
    conn.execute(q_create)

In [ ]:
%%time
with engine.connect() as conn:
    gdf_nodes[["x","y","osmid"]].rename(columns={"x":"eastings","y":"northings"}).to_sql("node_coords",index=False,con=conn,if_exists="append")